In [117]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/cw/hgl5p7pd36gfh8z1d954r9bw0000gn/T/ipykernel_60556/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [118]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [119]:
file_path = '/Users/marinacastilloariza/Desktop/AI_Work/4_Week/NLP/lab-natural-language-processing/data/kg_train.csv'
df = pd.read_csv(file_path)


- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [120]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("/Users/marinacastilloariza/Desktop/AI_Work/4_Week/NLP/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [121]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
data_train = train_data.copy()
data_test = test_data.copy()


## Data Preprocessing

In [122]:
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marinacastilloariza/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [123]:
!pip install beautifulsoup4 nltk
import re
from bs4 import BeautifulSoup


In [124]:
def clean_html(raw_html):
    soup = BeautifulSoup(raw_html, "html.parser")
    for script in soup(["script", "style"]):
        script.extract()
    return str(soup)

def remove_html_comments(cleaned_html):
    cleaned_html = re.sub(r'<!--.*?-->', '', cleaned_html, flags=re.DOTALL)
    
    return cleaned_html

def remove_html_tags(html):
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text()
    return text

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [125]:
import re

def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) 
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\b\w\b', '', text)
    text = re.sub(r'^\b\w\b\s*', '', text) 
    text = re.sub(r'\s+', ' ', text) 
    text = re.sub(r'^\b(b)', '', text)
    text = text.lower()
    text = text.strip()
    
    return text

data['cleaned_html'] = data['text'].apply(clean_html)
data['cleaned_html'] = data['cleaned_html'].apply(remove_html_comments)
data['cleaned_text'] = data['cleaned_html'].apply(remove_html_tags)


/var/folders/cw/hgl5p7pd36gfh8z1d954r9bw0000gn/T/ipykernel_60556/1538888025.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(raw_html, "html.parser")
/var/folders/cw/hgl5p7pd36gfh8z1d954r9bw0000gn/T/ipykernel_60556/1538888025.py:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(raw_html, "html.parser")
/var/folders/cw/hgl5p7pd36gfh8z1d954r9bw0000gn/T/ipykernel_60556/1538888025.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, "html.parser")
/var/folders/cw/hgl5p7pd36gfh8z1d954r9bw0000gn/T/ipykernel_60556/1538888025.py:13: MarkupResembl


Removing specific prefixes, like a prefixed 'b', is not a universal standard procedure in natural language processing (NLP). Instead, the decision to remove such prefixes depends on the specific context of the data being processed and the goals of the analysis. Here are some points to consider regarding the standard practices in NLP preprocessing:

Informal Language: In datasets containing informal language, such as text messages, tweets, or user-generated content, practitioners often choose to clean up non-standard usages (like slang or prefixes) to maintain the integrity of the analysis.
Formal Language: In more structured datasets (e.g., news articles or academic texts), such prefixes are less likely to appear, making this cleaning step unnecessary.

## Now let's work on removing stopwords
Remove the stopwords.

In [126]:
data['preprocessed_text'] = data['cleaned_text'].apply(clean_text)
nltk.download('stopwords')  # Ensure that stopwords are downloaded
stop_words = set(stopwords.words('english'))
data['final_text'] = data['preprocessed_text'].apply(
    lambda text: ' '.join(word for word in text.split() if word not in stop_words)
)
print(data[['text', 'preprocessed_text', 'final_text']].head())


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                   preprocessed_text  \
0  dear sir strictly private business proposal am...   
1                                            will do   
2  noracheryl has emailed dozens of memos about h...   
3  dear sirfmadamc know that this proposal might ...   
4                                                fyi   

                                          final_text  
0  dear sir strictly private business proposal mi...  
1                                                     
2  noracheryl emailed dozens memos haiti weekend ...  
3  dear sirfmadamc know proposal might surprise e...  
4                                                fy

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marinacastilloariza/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [127]:
from nltk.stem import PorterStemmer, SnowballStemmer, WordNetLemmatizer
porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
data['stemmed_text'] = data['final_text'].apply(
    lambda text: ' '.join(porter_stemmer.stem(word) for word in text.split())
)
nltk.download('wordnet') 
data['lemmatized_text'] = data['final_text'].apply(
    lambda text: ' '.join(lemmatizer.lemmatize(word) for word in text.split())
)
print(data[['text', 'preprocessed_text', 'final_text', 'stemmed_text', 'lemmatized_text']].head())


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/marinacastilloariza/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                   preprocessed_text  \
0  dear sir strictly private business proposal am...   
1                                            will do   
2  noracheryl has emailed dozens of memos about h...   
3  dear sirfmadamc know that this proposal might ...   
4                                                fyi   

                                          final_text  \
0  dear sir strictly private business proposal mi...   
1                                                      
2  noracheryl emailed dozens memos haiti weekend ...   
3  dear sirfmadamc know proposal might surprise e...   
4                                             

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [128]:
from sklearn.feature_extraction.text import CountVectorizer
ham_messages = data[data['label'] == 0]
spam_messages = data[data['label'] == 1]
vectorizer = CountVectorizer()

ham_word_counts = vectorizer.fit_transform(ham_messages['final_text'])
ham_word_sum = ham_word_counts.sum(axis=0)
ham_word_freq = pd.DataFrame(ham_word_sum.T, index=vectorizer.get_feature_names_out(), columns=['count']).sort_values(by='count', ascending=False)
top_10_ham_words = ham_word_freq.head(10)

spam_word_counts = vectorizer.fit_transform(spam_messages['final_text'])
spam_word_sum = spam_word_counts.sum(axis=0)
spam_word_freq = pd.DataFrame(spam_word_sum.T, index=vectorizer.get_feature_names_out(), columns=['count']).sort_values(by='count', ascending=False)
top_10_spam_words = spam_word_freq.head(10)

print("Top 10 Words in Ham Messages:")
print(top_10_ham_words)

print("\nTop 10 Words in Spam Messages:")
print(top_10_spam_words)

Top 10 Words in Ham Messages:
           count
pm           115
us           115
would        106
president     86
state         82
percent       77
call          75
work          67
mr            66
obama         62

Top 10 Words in Spam Messages:
             count
money          920
account        741
bank           725
us             550
business       474
fund           429
transaction    409
transfer       392
country        367
million        366


In [129]:
data_train['preprocessed_text'] = data_train['text'].apply(clean_text)


## Extra features

In [130]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_symbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"])

# Adding money mark and suspicious words
data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_symbol_list) * 1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words) * 1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x))

# Display the updated DataFrame
print(data_train.head())

                                                  text  label  \
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...      1   
535  I have not been able to reach oscar this am. W...      0   
695  ; Huma Abedin B6I'm checking with Pat on the 5...      0   
557  I can have it announced here on Monday - can't...      0   
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...      1   

                                     preprocessed_text  money_mark  \
29   regards mr nelson smithkindly reply me on my p...           1   
535  have not been able to reach oscar this am we a...           1   
695  huma abedin bim checking with pat on the will ...           1   
557    can have it announced here on monday cant today           1   
836  bank of africaagence san pedro bp san pedro co...           1   

     suspicious_words  text_len  
29                  0        85  
535                 0        91  
695                 0       128  
557                 0        47  
836               

## How would work the Bag of Words with Count Vectorizer concept?

In [131]:
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(data_train['preprocessed_text'])
X_train_array = X_train_counts.toarray()
feature_names = vectorizer.get_feature_names_out()
print("Shape of the Bag of Words matrix:", X_train_array.shape)  # (num_samples, num_features)
print("Feature names:", feature_names[:10])  # Display the first 10 words


Shape of the Bag of Words matrix: (800, 18350)
Feature names: ['aac' 'aaclocated' 'aae' 'aag' 'aaronovitchon' 'abacha' 'abachabefore'
 'abachac' 'abachace' 'abachaco']


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [132]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(data_train['preprocessed_text'])
X_train_tfidf_array = X_train_tfidf.toarray()
feature_names = tfidf_vectorizer.get_feature_names_out()
print("Shape of the TF-IDF matrix:", X_train_tfidf_array.shape)
print("Feature names:", feature_names[:10]) 

Shape of the TF-IDF matrix: (800, 18350)
Feature names: ['aac' 'aaclocated' 'aae' 'aag' 'aaronovitchon' 'abacha' 'abachabefore'
 'abachac' 'abachace' 'abachaco']


## And the Train a Classifier?

In [134]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
def preprocess_text(text):
    cleaned_text = clean_text(text)
    return cleaned_text
data['preprocessed_text'] = data['text'].apply(preprocess_text)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['preprocessed_text'])
X_test_tfidf = tfidf_vectorizer.transform(test_data['preprocessed_text'])
classifier = MultinomialNB()
classifier.fit(X_train_tfidf, train_data['label'])
predictions = classifier.predict(X_test_tfidf)
accuracy = accuracy_score(test_data['label'], predictions)

print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(test_data['label'], predictions))
print("Confusion Matrix:")
print(confusion_matrix(test_data['label'], predictions))


Accuracy: 0.84
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.74      0.85       125
           1       0.70      1.00      0.82        75

    accuracy                           0.84       200
   macro avg       0.85      0.87      0.84       200
weighted avg       0.89      0.84      0.84       200

Confusion Matrix:
[[93 32]
 [ 0 75]]


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).